In [ ]:
from shapely import geometry
import geopandas as gpd
import pandas as pd

## Data preparation


In order to use ```CountEmbedder``` we need to prepare some example data.  
Namely we need: ```regions_gdf```, ```features_gdf```, and ```joint_gdf```.  
NOTE: These are normally output by the previous pipeline steps.

In [ ]:
regions_gdf = gpd.GeoDataFrame(
    geometry=[
        geometry.Polygon(
            shell=[
                (17.02710946531851, 51.110065389823305),
                (17.029634931698617, 51.1092989279356),
                (17.03212452567607, 51.11021450606774),
                (17.032088692873092, 51.11189657169522),
                (17.029563145936592, 51.11266305206119),
                (17.02707351236059, 51.11174744831988),
                (17.02710946531851, 51.110065389823305)
            ],
        ),
        geometry.Polygon(
            shell=[
                (17.03212452567607, 51.11021450606774),
                (17.034649970341516, 51.109447934020366),
                (17.037139662738255, 51.11036340911803),
                (17.037103950094387, 51.11204548186887),
                (17.03457842489355, 51.11281207240022),
                (17.032088692873092, 51.11189657169522),
                (17.03212452567607, 51.11021450606774)
            ],
        ),
        geometry.Polygon(
            shell=[
                (17.02952725046974, 51.114345051613405),
                (17.029563145936592, 51.11266305206119),
                (17.032088692873092, 51.11189657169522),
                (17.03457842489355, 51.11281207240022),
                (17.03454264959235, 51.11449407907883),
                (17.03201702210393, 51.115260577927586),
                (17.02952725046974, 51.114345051613405)
            ],
        ),
    ],
    index = pd.Index(
        data=["891e2040897ffff", "891e2040d4bffff", "891e2040d5bffff"],
        name="region_id"
    ),
    crs="epsg:4326",
)
regions_gdf

In [ ]:
features_gdf = gpd.GeoDataFrame(
    {
    "leisure": ["playground", None, "adult_gaming_centre", None],
    "amenity": [None, "pub", "pub", None]
    },
    geometry=[
        geometry.Polygon(
            shell=[
                (17.0360858, 51.1103927),
                (17.0358804, 51.1104389),
                (17.0357855, 51.1105503),
                (17.0359451, 51.1105907),
                (17.0361589, 51.1105402),
                (17.0360858, 51.1103927),
            ]
        ),
        geometry.Polygon(
            shell=[
                (17.0317168, 51.1114868),
                (17.0320, 51.1114868),
                (17.0320, 51.1117503),
                (17.0317168, 51.1117503),
            ]
        ),
        geometry.Polygon(
            shell=[
                (17.0317168, 51.1124868),
                (17.0320, 51.1124868),
                (17.0320, 51.1127503),
                (17.0317168, 51.1127503),
            ]
        ),
        geometry.Polygon(
            shell=[
                (17.0307168, 51.1104868),
                (17.0310, 51.1104868),
                (17.0310, 51.1107503),
                (17.0307168, 51.1107503),
            ]
        ),
    ],
    index = pd.Index(
        data=["way/312457804", "way/1533817161", "way/312457812", "way/312457834"],
        name="feature_id"
    ),
    crs="epsg:4326",
)
features_gdf

In [ ]:
ax = regions_gdf.plot()
features_gdf.plot(ax=ax, color="red")

In [ ]:
joint_gdf = gpd.GeoDataFrame(
    geometry=[
        geometry.Polygon(
            shell=[
                (17.0358804, 51.1104389),
                (17.0357855, 51.1105503),
                (17.0359451, 51.1105907),
                (17.0361589, 51.1105402),
                (17.0360858, 51.1103927),
                (17.0358804, 51.1104389),
            ]
        ),
        geometry.Polygon(
            shell=[
                (17.0317168, 51.1117503),
                (17.032, 51.1117503),
                (17.032, 51.1114868),
                (17.0317168, 51.1114868),
                (17.0317168, 51.1117503),
            ]
        ),
        geometry.Polygon(
            shell=[
                (17.0307168, 51.1107503),
                (17.031, 51.1107503),
                (17.031, 51.1104868),
                (17.0307168, 51.1104868),
                (17.0307168, 51.1107503),
            ]
        ),
        geometry.Polygon(
            shell=[
                (17.0317168, 51.1127503),
                (17.032, 51.1127503),
                (17.032, 51.1124868),
                (17.0317168, 51.1124868),
                (17.0317168, 51.1127503),
            ]
        ),
    
    ],
    index = pd.MultiIndex.from_arrays(
        arrays=[
            ["891e2040d4bffff", "891e2040897ffff", "891e2040897ffff", "891e2040d5bffff"], 
            ["way/312457804", "way/1533817161", "way/312457834", "way/312457812"]
        ],
        names=["region_id", "feature_id"]
    ),
    crs="epsg:4326",
)
joint_gdf

In [ ]:
joint_gdf.plot()

## Embed using features existing in data

In [ ]:
from srai.embedders import CountEmbedder
embedder = CountEmbedder()

In [ ]:
embedding = embedder.embed(regions_gdf, features_gdf, joint_gdf)
embedding

## Embed with specifying expected output features

In [ ]:
from srai.embedders import CountEmbedder
embedder = CountEmbedder(expected_output_features=["amenity_parking", "leisure_park", "amenity_pub"])

In [ ]:
embedding_expected_features = embedder.embed(regions_gdf, features_gdf, joint_gdf)
embedding_expected_features

The resulting embedding contains only the columns specified in ```expected_output_features```.  
The ones that were not present in the data (```leisure_park```, ```amenity_parking```) are added and filled with zeros.   
The features that are both expected and present in the data are counted as usual.  
The ones that are present in the data but are not expected (```leisure_adult_gaming_centre```, ```leisure_playground```) are discarded.